<a href="https://colab.research.google.com/github/dukei/dls-agents/blob/main/DLS-project-agents-1_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Оценка релевантности организаций запросам на Яндекс.Картах с помощью LLM-агента

<img src="https://sun9-65.userapi.com/impg/N4y2cxlL7PauAs82tBNFOUAiNctFICWDy4Mbiw/Jiz1fb7NLWU.jpg?size=1080x1080&quality=95&sign=df2786058624d9ccac3ede4d5d056e2f&type=album" width="500" height="500" />


## Описание задачи

Необходимо создать LLM-агента, который будет оценивать релевантность организаций на Яндекс.Картах широким запросам (например, "ресторан с верандой" или "романтичный джаз-бар"). Такие запросы называются *рубричными*: пользователь здесь ищет не конкретную организацию, а идёт в Яндекс.Карты для поиска и выбора мест.

LLM-агент должен будет самостоятельно находить необходимые данные для принятия правильного решения.

Данные представлены компанией Яндекс и являются результатами асессорской разметки релевантности. Мы очень заинтересован в успешном решении задачи:)

## План работы

1. Написать сильный бейзлайн. Например, можно просто делать один запрос в LLM, возможно с добавлением в промпт размеченных примеров. Либо можно дообучить трансформер на задачу классификации.
2. Разобраться с фреймворком для реализации LLM-агентов (предлагается использовать LangGraph, оба примера в доп. материалах используют его).
3. Реализовать первую версию агента: предложить ему не принимать решение мгновенно, а ходить в поисковик для уточнения своего ответа.
4. Поработать над промптами для улучшения качества. Здесь нужно будет посмотреть, в каких запросах агент ошибается. **ПОЖАЛУЙСТА, НЕ ПОДГЛЯДЫВАЙТЕ, КАК ИМЕННО АГЕНТ ОШИБАЕТСЯ НА EVAL-МНОЖЕСТВЕ, ДЛЯ ЭТОГО ЕСТЬ TRAIN**.
5. Другие идеи (опционально):
- с помощью нейросетевого ретривала можно найти в обучающем множестве похожие размеченные примеры
- можно реализовать tool для парсинга сайта, которую агент будет вызывать

## Про платные API

Обратите внимание, что для реализации проекта потребуется самостоятельно ходить в OpenAI API (и другие платные API). На это может уйти существенная сумма (по грубой оценке ментора, 500 запусков агента на стеке OpenAI легко уложатся в 2к рублей, но риски вылезти за эту границу есть. Можно использовать и более дешевые LLM). Также поможет иностранная карта для оплаты этих сервисов, но в целом без нее можно обойтись.

Сам я для запросов в OpenAI API часто использую сервис https://vsegpt.ru/ (не реклама). При их использовании вам не нужен VPN, достаточно российской карты, а цены не сильно отличаются от цен OpenAI, но нужно оплатить подписку (400 рублей).

**О ТОМ, КАКИЕ КЛАССНЫЕ И ДЕШЕВЫЕ АЛЬТЕРНАТИВЫ ВЫ НАШЛИ, ОБЯЗАТЕЛЬНО НАПИШИТЕ В ЧАТ**

## Система оценивания

1. Реализация бейзлайна (без использования агента, но можно с использованием LLM) -- 2 балла
2. Реализация "какого-то" агента, который под капотом имеет возможность обращаться к поисковой строке для уточнения результата -- 4 балла
3. Существенно побить бейзлайн или безуспешно приложить к этому усилия и "показать", что агенты в этой задаче прироста не дают -- 4 балла
4. (опционально, за шоколадку) реализовать одну из доп. идей по улучшению агента

## Что почитать/посмотреть по теме

1. Статья, по которой можно составить какое-то представление об агенте -- https://habr.com/ru/articles/864646/
2. https://www.youtube.com/watch?v=U6LbW2IFUQw -- неплохое видео, где на практике подробно расписано, как агента можно создавать
3. Придётся самим много читать и разбираться. Делитесь идеями в чате:)

## Описание и загрузка данных

Данные: https://disk.yandex.ru/d/6d5hFHvpAZjQdw

Ваша задача -- предсказать колонку relevance, используя все остальные данные об организации. Загрузим данные и посмотрим на них

In [2]:
import requests

def download_file_from_yandex_disk(url, filename):
    base_url = "https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key="
    response = requests.get(base_url + url)
    download_link = response.json()['href']
    download_response = requests.get(download_link)
    with open(filename, 'wb') as f:
        f.write(download_response.content)
    print(f"File '{filename}' downloaded successfully.")

yandex_disk_url = "https://disk.yandex.ru/d/6d5hFHvpAZjQdw"
filename = "data_final_for_dls.jsonl"
download_file_from_yandex_disk(yandex_disk_url, filename)

File 'data_final_for_dls.jsonl' downloaded successfully.


In [3]:
import pandas as pd

data = pd.read_json(path_or_buf="/content/data_final_for_dls.jsonl", lines=True)
display(data.head())

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance,reviews_summarized
0,сигары,"Москва, Дубравная улица, 34/29",Tabaccos; Магазин Tabaccos; Табаккос,Магазин табака и курительных принадлежностей,1263329400,None,1.0,"Организация занимается продажей табака, курите..."
1,кальянная спб мероприятия,"Санкт-Петербург, Большой проспект Петроградско...",PioNero; Pionero; Пицца Паста бар; Pio Nero; P...,Кафе,228111266197,PioNero предлагает разнообразные блюда итальян...,0.0,"Организация PioNero — это кафе, бар и ресторан..."
2,Эпиляция,"Московская область, Одинцово, улица Маршала Жу...",MaxiLife; Центр красоты и здоровья MaxiLife; Ц...,Стоматологическая клиника,1247255817,"Стоматологическая клиника, массажный салон и к...",1.0,"Организация занимается стоматологическими, кос..."
3,спортзал где 1 занятие бесплатно,"Москва, Страстной бульвар, 13А",Унца Унца Спорт; Unza Unza Sport,Фитнес-клуб,201938477844,Фитнес-клуб предлагает пробные занятия по разл...,0.1,Организация «Унца Унца Спорт» предоставляет ус...
4,стиральных машин,"Москва, улица Обручева, 34/63",М.Видео; M Video; M. Видео; M.Видео; Mvideo; М...,Магазин бытовой техники,1074529324,М.Видео предлагает широкий ассортимент бытовой...,1.0,Организация занимается продажей бытовой техник...


In [7]:
data['relevance'].value_counts()

,count
relevance,
1.0,15882
0.0,14509
0.1,4703


Здесь 1.0 соответствует оценке RELEVANT_PLUS, 0.1 -- оценке RELEVANT_MINUS, 0.0 -- оценке IRRELEVANT.

Ваша задача -- построить LLM-агента, который будет предсказывать релевантность.

Выделим данные для оценки качества агента. Запуск агента -- это тяжелая и потенциально дорогая операция. Поэтому eval-множество имеет размер 500. Также для простоты из eval-множества выкинуты данные с оценкой RELEVANT_MINUS. Тем не менее, вы можете использовать такие примеры для подачи примеров агенту.

**ОБРАТИТЕ ВНИМАНИЕ, ЧТО В EVAL-ДАННЫЕ НЕЛЬЗЯ ПОДГЛЯДЫВАТЬ ДЛЯ КАЛИБРОВКИ АГЕНТА!!! ДЛЯ ЭТОГО ЕСТЬ ОБУЧАЮЩИЕ ДАННЫЕ**

В качестве метрики качества мы будем использовать обычную ACCURACY, поскольку классы сбалансированы.

In [8]:
train_data = data[570:]
eval_data = data[:570]
eval_data = eval_data[eval_data["relevance"] != 0.1]
eval_data

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance,reviews_summarized
0,сигары,"Москва, Дубравная улица, 34/29",Tabaccos; Магазин Tabaccos; Табаккос,Магазин табака и курительных принадлежностей,1263329400,None,1.0,"Организация занимается продажей табака, курите..."
1,кальянная спб мероприятия,"Санкт-Петербург, Большой проспект Петроградско...",PioNero; Pionero; Пицца Паста бар; Pio Nero; P...,Кафе,228111266197,PioNero предлагает разнообразные блюда итальян...,0.0,"Организация PioNero — это кафе, бар и ресторан..."
2,Эпиляция,"Московская область, Одинцово, улица Маршала Жу...",MaxiLife; Центр красоты и здоровья MaxiLife; Ц...,Стоматологическая клиника,1247255817,"Стоматологическая клиника, массажный салон и к...",1.0,"Организация занимается стоматологическими, кос..."
4,стиральных машин,"Москва, улица Обручева, 34/63",М.Видео; M Video; M. Видео; M.Видео; Mvideo; М...,Магазин бытовой техники,1074529324,М.Видео предлагает широкий ассортимент бытовой...,1.0,Организация занимается продажей бытовой техник...
5,сеть быстрого питания,"Санкт-Петербург, 1-я Красноармейская улица, 15",Rostic's; KFC; Ресторан быстрого питания KFC,Быстрое питание,1219173871,Rostic's предлагает различные наборы быстрого ...,1.0,"Организация занимается быстрым питанием, предо..."
...,...,...,...,...,...,...,...,...
561,наращивание ресниц,"Саратов, улица имени А.С. Пушкина, 1",Сила; Sila; Beauty brow; Студия бровей Beauty ...,Салон красоты,236976975812,Салон красоты «Сила» предлагает услуги по уход...,1.0,Организация «Сила» занимается предоставлением ...
565,игры,"Москва, Щёлковское шоссе, 79, корп. 1",YouPlay; YouPlay КиберКлуб,Компьютерный клуб,109673025161,YouPlay КиберКлуб предлагает услуги по игре на...,0.0,Организация занимается предоставлением услуг к...
566,домашний интернет в курске что подключить отзы...,"Курск, Садовая улица, 5",Цифровой канал; Digital Channel; DChannel; ЦК;...,Телекоммуникационная компания,1737991898,None,0.0,None
567,гостиница волгодонск сауна номер телефона,"Ростовская область, городской округ Волгодонск...",Поплавок; Poplavok,"База , дом отдыха",147783493467,"Предлагает размещение в различных типах жилья,...",0.0,Организация «Поплавок» предлагает услуги базы ...


In [9]:
# eval_data.to_excel("eval_data.xlsx")

### 1. Подготовка данных для бейзлайна

Выберем необходимые колонки из `eval_data`.

In [39]:
baseline_data = eval_data[['Text', 'address', 'name', 'normalized_main_rubric_name_ru', 'prices_summarized', 'reviews_summarized', 'relevance']].copy()
display(len(baseline_data), baseline_data.head())

500

,Text,address,name,normalized_main_rubric_name_ru,prices_summarized,reviews_summarized,relevance
0,сигары,"Москва, Дубравная улица, 34/29",Tabaccos; Магазин Tabaccos; Табаккос,Магазин табака и курительных принадлежностей,None,"Организация занимается продажей табака, курите...",1.0
1,кальянная спб мероприятия,"Санкт-Петербург, Большой проспект Петроградско...",PioNero; Pionero; Пицца Паста бар; Pio Nero; P...,Кафе,PioNero предлагает разнообразные блюда итальян...,"Организация PioNero — это кафе, бар и ресторан...",0.0
2,Эпиляция,"Московская область, Одинцово, улица Маршала Жу...",MaxiLife; Центр красоты и здоровья MaxiLife; Ц...,Стоматологическая клиника,"Стоматологическая клиника, массажный салон и к...","Организация занимается стоматологическими, кос...",1.0
4,стиральных машин,"Москва, улица Обручева, 34/63",М.Видео; M Video; M. Видео; M.Видео; Mvideo; М...,Магазин бытовой техники,М.Видео предлагает широкий ассортимент бытовой...,Организация занимается продажей бытовой техник...,1.0
5,сеть быстрого питания,"Санкт-Петербург, 1-я Красноармейская улица, 15",Rostic's; KFC; Ресторан быстрого питания KFC,Быстрое питание,Rostic's предлагает различные наборы быстрого ...,"Организация занимается быстрым питанием, предо...",1.0


### 2. Инициализация клиента Deepseek LLM

Для работы с Deepseek LLM (предполагая совместимость с OpenAI API) нам потребуется установить библиотеку `openai`.

Будем хранить свой API ключ в секретах Colab под именем `DEEPSEEK_API_KEY` и базовый URL под именем `DEEPSEEK_BASE_URL`.

In [11]:
!pip install openai

In [28]:
import os
from openai import AsyncOpenAI # Изменяем на AsyncOpenAI
from google.colab import userdata

# Получение API ключа и базового URL из секретов Colab
DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')
DEEPSEEK_BASE_URL = userdata.get('DEEPSEEK_BASE_URL') # Например, "https://api.deepseek.com/v1"

# Инициализация асинхронного клиента Deepseek
client = AsyncOpenAI(api_key=DEEPSEEK_API_KEY, base_url=DEEPSEEK_BASE_URL)

print("Асинхронный клиент Deepseek LLM инициализирован. Убедитесь, что DEEPSEEK_API_KEY и DEEPSEEK_BASE_URL установлены в секретах Colab.")

Асинхронный клиент Deepseek LLM инициализирован. Убедитесь, что DEEPSEEK_API_KEY и DEEPSEEK_BASE_URL установлены в секретах Colab.


### 3. Определение промпта для LLM для оценки релевантности

Создадим функцию, которая будет формировать промпт для LLM, используя данные об организации и запрос пользователя, и ожидать ответа в виде числовой оценки релевантности (0.0, 0.1 или 1.0).

Желательно побольше константных данных засунуть в системный промт, чтобы использовать кешировние токенов и сэкономить

In [19]:
def create_relevance_prompt(row):
    query = row['Text']
    org_name = row['name'] if pd.notna(row['name']) else 'Не указано'
    org_address = row['address'] if pd.notna(row['address']) else 'Не указан'
    org_rubric = row['normalized_main_rubric_name_ru'] if pd.notna(row['normalized_main_rubric_name_ru']) else 'Не указана'
    org_prices = row['prices_summarized'] if pd.notna(row['prices_summarized']) else 'Информация о ценах отсутствует.'
    org_reviews = row['reviews_summarized'] if pd.notna(row['reviews_summarized']) else 'Отзывы отсутствуют.'

    prompt = {
        "system": """Оцени релевантность организации запросу пользователя. Ответь только числом: 1.0, 0.1 или 0.0. 1.0 означает - полная релевантность, 0.1 - частичная, 0.0 - полностью нерелевантна.

Запрос пользователя: """,
        "user":  f""" "{query}"

Информация об организации:
Название: {org_name}
Адрес: {org_address}
Рубрика: {org_rubric}
Сводка цен: {org_prices}
Сводка отзывов: {org_reviews}
"""
    }

    return prompt

# Пример промпта для первой строки baseline_data
example_prompt = create_relevance_prompt(baseline_data.iloc[0])
print(example_prompt)

{'system': 'Оцени релевантность организации запросу пользователя. Ответь только числом: 1.0, 0.1 или 0.0. 1.0 означает - полная релевантность, 0.1 - частичная, 0.0 - полностью нерелевантна.\n\nЗапрос пользователя: ', 'user': ' "налоговая 5007"\n\nИнформация об организации:\nНазвание: Налоговая служба; Межрайонная ИФНС № 2; ИФНС № 2; Межрайонная ИФНС № 2 по Московской области; Межрайонная инспекция Федеральной налоговой службы № 2 по Московской области; Межрайонная ИФНС России № 2 по Московской области\nАдрес: Московская область, Королёв, улица Богомолова, 4\nРубрика: Налоговая инспекция\nСводка цен: Информация о ценах отсутствует.\nСводка отзывов: Организация занимается обслуживанием налогоплательщиков, предоставляя консультации и помощь в оформлении документов. Тональность отзывов смешанная: много положительных отзывов о вежливости и компетентности сотрудников, но также есть критические замечания о безграмотности и грубом отношении. Хвалят: отзывчивость и профессионализм сотрудников, 

### 4. Получение предсказаний от Deepseek LLM

Теперь мы будем итерироваться по `baseline_data`, создавать промпты и отправлять их в Deepseek LLM для получения предсказаний релевантности.

Причем, чтобы это не занимало слишком много времени, будем делать это по возможности параллельно в несколько потоков.

In [36]:
# Применение функции к baseline_data
# Внимание: Это может занять много времени и потребовать затрат на API
# Для начала можно протестировать на небольшой выборке

import asyncio
import pandas as pd

async def get_deepseek_prediction_async(prompt):
    try:
        response = await client.chat.completions.create(
            model="deepseek-chat", # Укажите используемую модель Deepseek
            messages=[
                {"role": "system", "content": prompt['system']},
                {"role": "user", "content": prompt['user']},
            ],
            temperature=0.0, # Для воспроизводимости и получения детерминированного ответа
            max_tokens=5 # Ожидаем короткий ответ: 1.0, 0.1 или 0.0
        )
        prediction_str = response.choices[0].message.content.strip()

        # Попытка преобразовать ответ в float
        try:
            prediction_float = float(prediction_str)
            if prediction_float in [0.0, 0.1, 1.0]:
                return prediction_float
            else:
                print(f"Предупреждение: LLM вернул неожиданное значение '{prediction_str}'. Возвращаю 0.0.")
                return 0.0
        except ValueError:
            print(f"Предупреждение: Не удалось преобразовать '{prediction_str}' в float. Возвращаю 0.0.")
            return 0.0
    except Exception as e:
        print(f"Ошибка при вызове Deepseek API: {e}. Возвращаю 0.0.")
        return 0.0

async def get_predictions_concurrently(data, concurrency=5):
    predictions = []
    semaphore = asyncio.Semaphore(concurrency)

    async def process_row(row):
        async with semaphore:
            prompt = create_relevance_prompt(row)
            return await get_deepseek_prediction_async(prompt)

    tasks = [process_row(row) for index, row in data.iterrows()]
    predictions = await asyncio.gather(*tasks)
    return predictions

# Установим лимит для демонстрации
limit = 500 # Обработаем полную выборку

# Запуск параллельных предсказаний
predictions_list = await get_predictions_concurrently(baseline_data.head(limit), concurrency=25)
baseline_data.loc[baseline_data.head(limit).index, 'predicted_relevance'] = predictions_list

display(baseline_data.head(limit)[['Text', 'relevance', 'predicted_relevance']])


,Text,relevance,predicted_relevance
0,сигары,1.0,0.1
1,кальянная спб мероприятия,0.0,0.0
2,Эпиляция,1.0,1.0
4,стиральных машин,1.0,1.0
5,сеть быстрого питания,1.0,1.0
...,...,...,...
561,наращивание ресниц,1.0,1.0
565,игры,0.0,1.0
566,домашний интернет в курске что подключить отзы...,0.0,0.1
567,гостиница волгодонск сауна номер телефона,0.0,0.1


### 5. Оценка производительности бейзлайна

Сравним предсказания LLM с истинными значениями релевантности и рассчитаем точность.

In [37]:
from sklearn.metrics import accuracy_score
import numpy as np

# Отфильтруем строки, для которых были получены предсказания
predicted_data = baseline_data.head(limit).dropna(subset=['predicted_relevance'])

if not predicted_data.empty:
    true_labels = predicted_data['relevance'].values
    predicted_labels = predicted_data['predicted_relevance'].values

    # Преобразуем метки с плавающей точкой в целые числа для метрик классификации
    # Сопоставление: 0.0 -> 0, 0.1 -> 1, 1.0 -> 2
    # Это предполагает, что это отдельные категории, а не непрерывные значения.
    # Более безопасный подход для этих конкретных значений — сопоставить их с уникальными целыми числами.
    # Сопоставим 0.0 с 0, 0.1 с 1 и 1.0 с 2 для дискретной классификации.
    mapping = {0.0: 0, 0.1: 1, 1.0: 2}
    true_labels_mapped = np.array([mapping[val] for val in true_labels])
    predicted_labels_mapped = np.array([mapping[val] for val in predicted_labels])

    accuracy = accuracy_score(true_labels_mapped, predicted_labels_mapped)
    print(f"Точность бейзлайн-модели на {len(predicted_data)} примерах: {accuracy:.4f}")
else:
    print("Нет данных для оценки точности. Убедитесь, что были получены предсказания.")

Точность бейзлайн-модели на 500 примерах: 0.4520


Сохраним бейзлайн оценку в csv, чтобы было с чем сравнивать в будущем

### 6. Итоговое резюме

Кратко подведем итоги работы бейзлайна Deepseek LLM и сделаем выводы.

Бейзлайн (использование LLM без функций агента) показывает точность **0.4520**. Это совершенно немного, хотя и больше, чем случайное попадание (уровень 0.33 на трех сбалансированных классах). Попробуем далее превозмочь этот уровень.

In [43]:
predicted_data.to_csv("baseline_predictions.csv", index=False)
print("Содержимое baseline_data сохранено в файл baseline_predictions.csv")

Содержимое baseline_data сохранено в файл baseline_predictions.csv


In [42]:
display(predicted_data.head(limit))

,Text,address,name,normalized_main_rubric_name_ru,prices_summarized,reviews_summarized,relevance,predicted_relevance
0,сигары,"Москва, Дубравная улица, 34/29",Tabaccos; Магазин Tabaccos; Табаккос,Магазин табака и курительных принадлежностей,None,"Организация занимается продажей табака, курите...",1.0,0.1
1,кальянная спб мероприятия,"Санкт-Петербург, Большой проспект Петроградско...",PioNero; Pionero; Пицца Паста бар; Pio Nero; P...,Кафе,PioNero предлагает разнообразные блюда итальян...,"Организация PioNero — это кафе, бар и ресторан...",0.0,0.0
2,Эпиляция,"Московская область, Одинцово, улица Маршала Жу...",MaxiLife; Центр красоты и здоровья MaxiLife; Ц...,Стоматологическая клиника,"Стоматологическая клиника, массажный салон и к...","Организация занимается стоматологическими, кос...",1.0,1.0
4,стиральных машин,"Москва, улица Обручева, 34/63",М.Видео; M Video; M. Видео; M.Видео; Mvideo; М...,Магазин бытовой техники,М.Видео предлагает широкий ассортимент бытовой...,Организация занимается продажей бытовой техник...,1.0,1.0
5,сеть быстрого питания,"Санкт-Петербург, 1-я Красноармейская улица, 15",Rostic's; KFC; Ресторан быстрого питания KFC,Быстрое питание,Rostic's предлагает различные наборы быстрого ...,"Организация занимается быстрым питанием, предо...",1.0,1.0
...,...,...,...,...,...,...,...,...
561,наращивание ресниц,"Саратов, улица имени А.С. Пушкина, 1",Сила; Sila; Beauty brow; Студия бровей Beauty ...,Салон красоты,Салон красоты «Сила» предлагает услуги по уход...,Организация «Сила» занимается предоставлением ...,1.0,1.0
565,игры,"Москва, Щёлковское шоссе, 79, корп. 1",YouPlay; YouPlay КиберКлуб,Компьютерный клуб,YouPlay КиберКлуб предлагает услуги по игре на...,Организация занимается предоставлением услуг к...,0.0,1.0
566,домашний интернет в курске что подключить отзы...,"Курск, Садовая улица, 5",Цифровой канал; Digital Channel; DChannel; ЦК;...,Телекоммуникационная компания,None,None,0.0,0.1
567,гостиница волгодонск сауна номер телефона,"Ростовская область, городской округ Волгодонск...",Поплавок; Poplavok,"База , дом отдыха","Предлагает размещение в различных типах жилья,...",Организация «Поплавок» предлагает услуги базы ...,0.0,0.1
